In [8]:
from collections import Counter
def exact_match(house_trades, street_trades):
    house_counts = Counter(house_trades)
    street_counts = Counter(street_trades)
    trades = set()
    trades.update(house_counts.keys())
    trades.update(street_counts.keys())
    res = {}

    for trade in trades:
        if trade in house_counts and trade in street_counts:
            matched = min(house_counts[trade], street_counts[trade])
            house_counts[trade] -= matched
            street_counts[trade] -= matched
            res[trade] = abs(house_counts[trade] - street_counts[trade])
        elif trade in house_counts and trade not in street_counts:
            res[trade] = house_counts[trade]
        else:
            res[trade] = street_counts[trade]
    
    res = {key: count for key, count in sorted(res.items(), key=lambda item: item[0].strip())}
    res_list = []
    for key, count in res.items():
        res_list.extend([key for _ in range(count)])

    return res_list

In [9]:
house_trade_0 = ["AAPL,B,0100,ABC123","AAPL,B,0100,ABC123","AAPL,B,0100,ABC123","GOOG,S,0050,CDC333"]
street_trade_0 = [ " FB,B,0100,GBGGGG","AAPL,B,0100,ABC123"]

exact_match(house_trade_0, street_trade_0)


['AAPL,B,0100,ABC123',
 'AAPL,B,0100,ABC123',
 ' FB,B,0100,GBGGGG',
 'GOOG,S,0050,CDC333']

In [ ]:
from collections import defaultdict
def exact_match(house_counts, street_counts):
    for trade in trades:
        if trade in house_counts and trade in street_counts:
            matched = min(house_counts[trade], street_counts[trade])
            house_counts[trade] -= matched
            street_counts[trade] -= matched
    

def fuzzy_match(house_trades, street_trades):
    house_counts = Counter(house_trades)
    street_counts = Counter(street_trades)
    trades = set()
    trades.update(house_counts.keys())
    trades.update(street_counts.keys())
    res = {}
    exact_match(house_trades, street_trades)
    fuzzy_house_trades = defaultdict(list)
    for trade, count in house_counts.items():
        if count != 0:
            key = trade[:-7]
            fuzzy_house_trades[key].extend([trade for _ in range(count)])
    fuzzy_street_trades = defaultdict(list)
    for trade, count in street_counts.items():
        if count != 0:
            key = trade[:-7]
            fuzzy_street_trades[key].extend([trade for _ in range(count)])
    
    # house: "AAPL,B,0100,BCD123", "AAPL,B,0100,ABC123",
    # street: "AAPL,B,0100,XYZ123",
    # fuzzy_house_trades = {"AAPL,B,0100": ["AAPL,B,0100,BCD123", "AAPL,B,0100,ABC123"]}
    # 
    for trade in fuzzy_house_trades.keys():
        if trade in fuzzy_street_trades:
            matched = min(len(fuzzy_house_trades[trade]), len(fuzzy_street_trades[trade]))

            fuzzy_street_trades[trade] = sorted(fuzzy_street_trades[trade])[matched:]
            fuzzy_house_trades[trade] = sorted(fuzzy_house_trades[trade])[matched:]


In [ ]:
def fuzzy_match(house_trades, street_trades):
    house_trades.sort()
    street_trades.sort()
    hi, si = 0, 0
    

In [45]:
from collections import Counter
class Match:
    def __init__(self, house_trades, street_trades):
        self.house_cnt = Counter(house_trades)
        self.street_cnt = Counter(street_trades)
        print("initial house", self.house_cnt)
        print("initial street", self.street_cnt)

    def compose_result(self):
        res = []
        for trade, cnt in self.house_cnt.items():
            if cnt != 0:
                res.append(trade)
        for trade, cnt in self.street_cnt.items():
            if cnt != 0:
                res.append(trade)
        return sorted(res)


    def exact_match(self):
        for house, hc in sorted(self.house_cnt.items()):
            for street, sc in sorted(self.street_cnt.items()):
                if house == street:
                    to_match = min(hc, sc)
                    self.house_cnt[house] -= to_match
                    self.street_cnt[street] -= to_match
                    continue
        print("after exact", self.house_cnt)
        print("after exact", self.street_cnt)
    
    def fuzzy_match(self):
        def fuzzy_equal(s1, s2):
            s1_list = s1.split(",")[:-1]
            s1_no_id = ",".join(s1_list)
            s2_list = s2.split(",")[:-1]
            s2_no_id = ",".join(s2_list)
            return s1_no_id == s2_no_id
        
        for house, hc in sorted(self.house_cnt.items()):
            for street, sc in sorted(self.street_cnt.items()):
                if fuzzy_equal(house, street):
                    to_match = min(hc, sc)
                    self.house_cnt[house] -= to_match
                    self.street_cnt[street] -= to_match

        print("after fuzzy", self.house_cnt)
        print("after fuzzy", self.street_cnt)

    def offset_match(self):
        def offset_equal(s1, s2):
            s1_list = s1.split(",")
            s1_offset = s1_list[0] + "," + s1_list[2]
            s1_type = s1_list[1]
            s2_list = s2.split(",")
            s2_offset = s2_list[0] + "," + s2_list[2]
            s2_type = s2_list[1]
            return s1_type != s2_type and s1_offset == s2_offset
        
        def run_offset(cnt):
            keys = list(cnt.keys())
            for i in range(len(keys)):
                for j in range(i+1, len(keys)):
                    if offset_equal(keys[i], keys[j]):
                        to_match = min(cnt[keys[i]], cnt[keys[j]])
                        cnt[keys[i]] -= to_match
                        cnt[keys[j]] -= to_match
        run_offset(self.house_cnt)
        run_offset(self.street_cnt)
        print("after offset", self.house_cnt)
        print("after offset", self.street_cnt)




In [46]:
def match(house_trades, street_trades, match_type):
    m = Match(house_trades, street_trades)

    if match_type=="exact":
        m.exact_match()
    elif match_type=="fuzzy":
        m.exact_match()
        m.fuzzy_match()
    elif match_type=="offset":
        m.exact_match()
        m.fuzzy_match()
        m.offset_match()
    return m.compose_result()
     
    
# house_trades_0 = ["AAPL,B,0100,ABC123", "GOOG,S,0050,CDC333","AAPL,S,0100,ABE123","AAPL,B,0100,ABC123"]
# street_trades_0 = ["  FB,B,0100,GBGGGG", "AAPL,B,0100,ABS123"]
# match(house_trades_0, street_trades_0, "exact")
# match(house_trades_0, street_trades_0, "fuzzy")
# match(house_trades_0, street_trades_0, "offset")

# house_trades_1 = ["AAPL,S,0010,ZYX445", "AAPL,S,0010,ZYX446", "AAPL,B,0010,ABC123", "GOOG,S,0050,GHG545"]
# street_trades_1 = ["GOOG,S,0050,GHG545", "AAPL,S,0010,ZYX446", "AAPL,B,0010,ZYX445"]
# match(house_trades_1, street_trades_1, "exact")
# match(house_trades_1, street_trades_1, "fuzzy")
# match(house_trades_1, street_trades_1, "offset")

house_trades_2 = ["AAPL,S,0010,ZYX445", "AAPL,S,0019,ZYX446", "AAPL,B,0010,ABC123", "GOOG,S,0050,GHG545", "AAPL,B,0019,ZYX456"]
street_trades_2 = ["GOOG,S,0050,GHG545", "AAPL,S,0010,ZYX444", "AAPL,B,0010,TTT222"]
match(house_trades_2, street_trades_2, "exact")
match(house_trades_2, street_trades_2, "fuzzy")
match(house_trades_2, street_trades_2, "offset")


initial house Counter({'AAPL,S,0010,ZYX445': 1, 'AAPL,S,0019,ZYX446': 1, 'AAPL,B,0010,ABC123': 1, 'GOOG,S,0050,GHG545': 1, 'AAPL,B,0019,ZYX456': 1})
initial street Counter({'GOOG,S,0050,GHG545': 1, 'AAPL,S,0010,ZYX444': 1, 'AAPL,B,0010,TTT222': 1})
after exact Counter({'AAPL,S,0010,ZYX445': 1, 'AAPL,S,0019,ZYX446': 1, 'AAPL,B,0010,ABC123': 1, 'AAPL,B,0019,ZYX456': 1, 'GOOG,S,0050,GHG545': 0})
after exact Counter({'AAPL,S,0010,ZYX444': 1, 'AAPL,B,0010,TTT222': 1, 'GOOG,S,0050,GHG545': 0})
initial house Counter({'AAPL,S,0010,ZYX445': 1, 'AAPL,S,0019,ZYX446': 1, 'AAPL,B,0010,ABC123': 1, 'GOOG,S,0050,GHG545': 1, 'AAPL,B,0019,ZYX456': 1})
initial street Counter({'GOOG,S,0050,GHG545': 1, 'AAPL,S,0010,ZYX444': 1, 'AAPL,B,0010,TTT222': 1})
after exact Counter({'AAPL,S,0010,ZYX445': 1, 'AAPL,S,0019,ZYX446': 1, 'AAPL,B,0010,ABC123': 1, 'AAPL,B,0019,ZYX456': 1, 'GOOG,S,0050,GHG545': 0})
after exact Counter({'AAPL,S,0010,ZYX444': 1, 'AAPL,B,0010,TTT222': 1, 'GOOG,S,0050,GHG545': 0})
after fuzzy Co

[]

In [12]:
a = {}
a["a"] = 4
a["c"] = 3
a["b"] = 2
print(a)
print(sorted(a.items()))

{'a': 4, 'c': 3, 'b': 2}
[('a', 4), ('b', 2), ('c', 3)]
